In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup

In [ ]:
dataframe = pd.read_csv("css_dataset.csv")
dataframe.tail()

In [ ]:
dataframe['status'] = dataframe['status'].apply(lambda x: 1 if x=='h1' else 0)

In [ ]:
from collections import Counter
print(Counter(dataframe['status']))

In [ ]:
dataframe.tail()

In [ ]:
dataframe.shape

In [ ]:
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("status")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
from tensorflow.keras.layers import IntegerLookup
from tensorflow.keras.layers import Normalization
from tensorflow.keras.layers import StringLookup
from keras.layers import Embedding
from keras.layers import concatenate



def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature


def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = StringLookup if is_string else IntegerLookup
    # Create a lookup layer which will turn strings into integer indices
    lookup = lookup_class(output_mode="binary")

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the set of possible string values and assign them a fixed integer index
    lookup.adapt(feature_ds)

    # Turn the string input into integer indices
    encoded_feature = lookup(feature)
    return encoded_feature


In [ ]:
centre_freq = keras.Input(shape=(1,), name="centre_freq", dtype="int64")
freq = keras.Input(shape=(1,), name="freq", dtype="int64")


# Numerical features
power_db= keras.Input(shape=(1,), name="power_db")
noise_floor_db = keras.Input(shape=(1,), name="noise_floor_db")


all_inputs = [
              centre_freq,
              freq,
              power_db,
              noise_floor_db 
]

# Integer categorical features
centre_freq_encoded = encode_categorical_feature(centre_freq, "centre_freq", train_ds, False)
freq_encoded = encode_categorical_feature(freq, "freq", train_ds, False)


# Numerical features
power_db_encoded = encode_numerical_feature(power_db, "power_db", train_ds)
noise_floor_db_encoded = encode_numerical_feature(noise_floor_db, "noise_floor_db", train_ds)


all_features = layers.concatenate(
     [
      centre_freq_encoded, 
      freq_encoded, 
      power_db_encoded, 
      noise_floor_db_encoded,
      ])


# concat_feature_layer = layers.Concatenate(axis=4)([centre_freq_encoded, inceptionV3_copy, inception_resnet152_copy])
# all_feature = concatenate() ([layers], [centre_freq_encoded, freq_encoded, power_db_encoded, noise_floor_db_encoded])

# print(all_features)
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)


output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.compile(optimizer='adam',loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])



In [ ]:
from keras.models import load_model
model.save_weights('my_model',save_format='tf')  # creates a HDF5 file 'my_model.h5'
 # deletes the existing model

# returns am identical compiled model
#model = model.load_weights('my_model.h5')
#model.save_weights(filepath='final_weight.h5')

In [ ]:
model = model.load_weights('my_model')

In [ ]:
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")
history = model.fit(train_ds, epochs=10, validation_data=val_ds)
sample = {
    "centre_freq":24,
    "freq": 149954625.0,
    "power_db": 1500031250,
     "noise_floor_db": 56,
}

input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
predictions = model.predict(input_dict)
np.argmax(predictions,axis=1)
print(
    "This spectrum band  had a %.1f percent probability "
    "of having a spectrum hole(Primary User absent probability), as evaluated by our model." % (100 * predictions[0][0],)
)

Epoch 1/10
5353/5353 [==============================] - 121s 22ms/step - loss: 3.2613e-05 - accuracy: 1.0000 - val_loss: 5.0463e-04 - val_accuracy: 0.9997
Epoch 2/10
5353/5353 [==============================] - 101s 19ms/step - loss: 5.2527e-05 - accuracy: 1.0000 - val_loss: 3.6543e-04 - val_accuracy: 0.9998
Epoch 3/10
5353/5353 [==============================] - 99s 18ms/step - loss: 3.7593e-05 - accuracy: 1.0000 - val_loss: 3.9181e-04 - val_accuracy: 0.9998
Epoch 4/10
5353/5353 [==============================] - 99s 18ms/step - loss: 8.1449e-05 - accuracy: 1.0000 - val_loss: 2.9103e-04 - val_accuracy: 0.9999
Epoch 5/10
5353/5353 [==============================] - 101s 19ms/step - loss: 1.8778e-05 - accuracy: 1.0000 - val_loss: 2.4882e-04 - val_accuracy: 0.9999
Epoch 6/10
5353/5353 [==============================] - 100s 19ms/step - loss: 2.1370e-05 - accuracy: 1.0000 - val_loss: 3.1742e-04 - val_accuracy: 0.9998
Epoch 7/10
5353/5353 [==============================] - 100s 19ms/step -

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['accuracy']
epochs = range(1,51)
# plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs,loss_val , 'b', label='accuracy')
plt.plot(epochs,epochs , 'r', label='Epoch')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
# plt.show()

In [ ]:
import numpy as np
